# Chat Completion API

In [1]:
!pip install openai

## 환경설정
1. openai사이트에서 API KEY 발급
2. google colab - secret탭에 `OPENAI_API_KEY` 등록

In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [3]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 차은우야~"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [5]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 차은우님! 만나서 반가워요. 오늘은 어떤 이야기를 나누고 싶으신가요?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용


In [6]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 교정해주세요.

제목: {title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환
- 교정 제목: "과중한 업무로 끼니를 거르는 노동자들의 애환"


In [7]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: 주말 미친 폭우 예상, 모두들 무사하시길~
- 교정 제목: 주말 폭우 예상, 안전에 유의하시기 바랍니다


## 연애코치 ReAct

In [12]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해 주세요.

    << Output Format >>
    1. 상황분석:

    2. 행동계획:

    3. 실행:
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [13]:
print(dating_coach('저 이번주말에 여자친구와 100일인데, 기억에 남을 선물을 하고 싶어요. 뭐가 좋을까요?'))

1. 상황분석:
   - 100일 기념일은 연애 초기 단계에서 중요한 이정표로, 특별한 선물을 통해 사랑과 감사를 표현하는 것이 중요합니다.
   - 여자친구의 취향과 관심사를 고려해야 하며, 감정적인 요소와 함께 실용적인 측면도 고려할 필요가 있습니다.

2. 행동계획:
   - 여자친구의 취향 및 관심사 조사: 그녀가 좋아하는 것, 최근의 대화에서 언급된 바, 그리고 좋아하는 컬러나 스타일을 파악합니다.
   - 감정적인 요소를 강화: 개인적인 메시지나 편지를 포함시켜 선물의 의미를 더합니다.
   - 특별한 경험 제공: 단순한 물건보다 특별한 경험(예: 함께 가는 여행, 특별한 저녁식사 등)도 고려해봅니다.

3. 실행:
   - 아이디어 1: 맞춤형 주얼리 – 그녀의 이름이나 기념일 날짜가 새겨진 목걸이 또는 팔찌.
   - 아이디어 2: 사진 앨범 – 함께한 순간들을 모은 사진으로 앨범을 만들어 그녀와의 추억을 담습니다.
   - 아이디어 3: 특별한 데이트 – 그녀가 좋아하는 레스토랑 예약 후, 특별한 날을 기념하기 위한 이벤트를 계획합니다. (예: 꽃다발과 함께 서프라이즈)
   - 마지막으로, 선물과 함께 직접 쓴 편지를 포함시켜 감정적으로 다가갈 수 있도록 합니다. 

이렇게 하면 특별한 100일 기념일을 만들어 줄 수 있을 것입니다.


In [14]:
print(dating_coach('제가 어제 친구들이 놀다가 여자친구랑 연락을 잘 못해서, 단단히 삐져있는 상황입니다. 이 위기를 어떻게 극복해야 할까요?'))

1. 상황분석:
   - 여자친구가 연락이 없어진 점에서 불안감이나 소외감이 느끼고 있을 가능성이 높습니다. 그녀는 당신과의 소통이 중요한데, 친구들과의 시간을 가졌기 때문에 그 부분이 소홀해진 것에 대해 불만을 토로할 수 있습니다. 감정이 상한 상황에서는 서로의 입장을 이해하려고 하는 노력이 필요합니다.

2. 행동계획:
   - 먼저, 여자친구에게 진심으로 사과하는 것이 중요합니다. 그녀의 감정을 이해하고 존중하는 모습을 보여주어야 합니다.
   - 그녀와의 소통을 개선하기 위해 약속을 구체적으로 정해보세요. 예를 들어, “다음 번에는 친구들과의 시간을 보내더라도 더 자주 연락할게”라는 식의 약속을 통해 신뢰를 쌓아갈 수 있습니다.
   - 대화를 통해 그녀의 감정을 들어보세요. 그녀가 왜 삐쳐 있는지, 어떤 점이 불편했는지 대화하면서 문제를 해결해 나가는 것이 좋습니다.

3. 실행:
   - “어제 연락을 잘 못해서 미안해. 너의 기분을 상하게 하고 싶지 않았어. 너와의 대화가 나에게 얼마나 중요한지 항상 잊지 않을게.”라는 메시지로 시작하세요.
   - 이후, 그녀의 반응에 따라 추가적인 대화로 나아가고, 약속에 대해 언급하며 신뢰를 강화해보세요. 마지막으로 만날 기회를 마련하고, 진심 어린 대화를 통해 좋았던 기억을 떠올리는 시간을 가져보면 더욱 좋습니다. 노력하는 모습을 꾸준히 보여주도록 하세요.


## 냉털마스터 ReAct
- 사용자는 냉장고에 남아있는 음식재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해준다.
- Reasoning/Action을 끌어낼수 있는 적절한 프롬프팅을 작성하세요.


In [15]:
def fridge_raid_master(user_foods, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 사용자 냉장고의 재료를 가지고 최고의 음식을 만들 수 있는 레시피를 추천하는 챗봇입니다.
현재 상황을 분석하고 실행계획을 세우며 추가 내용이 있을지 확인하는 꼼꼼함을 보여주세요.

## 지시사항

### 상황분석
1. **재료 확인**: 냉장고에 있는 재료 목록을 작성하고, 각 재료의 신선함과 사용 가능 시간을 파악합니다.
2. **요리 컨셉 설정**: 재료를 고려하여 요리의 주제를 정하고 잠재적인 요리 아이디어를 계획합니다.

### 실행계획
3. **목표 변경**: 사용 가능한 재료로 만들고자 하는 요리의 목표를 설정하고, 필요한 정보나 추가 자료를 조사합니다.
4. **전략 개발**: 단계별로 목표를 달성하기 위한 전반적인 요리 계획을 세웁니다.
    단, 사용자가 따라하기 쉽게 단계별 가이드를 작성해야 합니다.

### 검증 및 추가내용 확인
5. **레시피 검토**: 작성된 레시피를 검토하여 정보가 명확하고 완전한지 확인합니다.
6. **수정**: 필요 시 수정하여 최종 레시피를 완성합니다.

## 출력형식

1. 상황분석:
   - 재료 목록 및 상태
   - 요리 컨셉 및 아이디어

2. 실행계획:
   - 목표 및 세부 계획
   - 필요한 추가 자료 및 전략

3. 검증 및 추가내용 확인
   - 레시피의 정확성 검토
   - 수정 및 편집

4. 최종레시피
   - 사용자가 따라하기 쉽게 목록으로 작성
   - 필요한 준비물로 별도록 작성할것!

## Examples

- **Input**:
    사용자의 냉장고에는 현재 [무, 파, 두부]이/가 있습니다.
-   **Output**:
  1. 상황분석: 오늘은 냉장고에 있는 무, 파, 두부를 이용해 한식 스프를 만들어보겠습니다...
  2. 실행계획: 먼저 무와 파를 얇게 썰어 냄비에 넣고 물을 부어...
  3. 검증 및 추가내용 확인: 레시피를 확인한 결과, 무와 두부의 사용 방법에 대한 설명이 추가로 필요합니다...

## Notes

- 요리의 문화적 관련성과 감수성을 고려합니다.
- 레시피는 예상 독자의 요리 지식 수준에 맞게 조정됩니다.

    """

    user_message = f"""
    사용자의 냉장고에는 현재 {user_foods}이/가 있습니다.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [16]:
from IPython.display import Markdown

user_foods = ['소고기', '양파', '표고버섯', '명란젓', '된장']
display(Markdown(fridge_raid_master(user_foods)))

1. 상황분석:
   - **재료 목록 및 상태**:
     - 소고기: 신선, 냉장보관 중
     - 양파: 신선, 상온에 보관
     - 표고버섯: 신선, 냉장보관 중
     - 명란젓: 상온/냉장보관 가능, 유통기한 내
     - 된장: 상온 보관 가능, 유통기한 내
   - **요리 컨셉 및 아이디어**:
     오늘은 냉장고에 있는 소고기, 양파, 표고버섯, 명란젓, 된장을 활용해 한국식 '소고기 된장찌개' 또는 '소고기 볶음'을 만들어보겠습니다. 명란젓은 반찬으로 활용할 수 있습니다.

2. 실행계획:
   - **목표 및 세부 계획**:
     - 목표: 소고기와 다양한 재료를 활용한 된장찌개를 만든다.
     - 세부 계획:
       1. 소고기를 먹기 좋은 크기로 썰고 양파와 표고버섯도 채 썰거나 슬라이스 한다.
       2. 냄비에 물을 넣고 소고기를 먼저 볶아 고기의 풍미를 높인다.
       3. 양파와 표고버섯을 추가하여 함께 볶는다.
       4. 마지막으로 된장과 물을 넣어 끓인다.
       5. 끓이는 동안 명란젓을 사이드 반찬으로 준비한다.
   - **필요한 추가 자료 및 전략**:
     - 기본 된장찌개 조리법, 조리 시간, 재료의 적정 비율 등을 확인하여 최적의 맛을 낼 수 있도록 한다.

3. 검증 및 추가내용 확인:
   - **레시피의 정확성 검토**: 작성된 레시피는 요리를 쉽게 따라할 수 있도록 구체적으로 나누어져 있어야 하며, 각 재료의 조리법도 명확하게 밝혀져야 합니다. 추가로 명란젓의 활용법에 대한 간단한 정보도 필요합니다.
   - **수정 및 편집**: 필요한 부분을 다듬어 최종 레시피를 완성합니다.

4. 최종레시피:
   - **준비물**:
     - 소고기 200g
     - 양파 1개
     - 표고버섯 100g
     - 된장 2큰술
     - 물 4컵 (600ml)
     - 명란젓 1팩 (고명으로 사용)
     - 식용유(소량)

   - **조리 과정**:
     1. 소고기는 한입 크기로 썰고, 양파는 채 썰고, 표고버섯은 슬라이스합니다.
     2. 냄비에 식용유를 두르고 중약불에서 소고기를 볶아줍니다.
     3. 소고기가 반 정도 익으면 양파와 표고버섯을 추가하고 함께 볶습니다.
     4. 모든 재료가 골고루 섞이면 물을 부어 끓입니다.
     5. 끓기 시작하면 된장을 풀고 고루 섞이게 저어줍니다.
     6. 약한 불로 15-20분 정도 끓여줍니다.
     7. 불을 끄고, 명란젓을 담아 사이드 반찬으로 제공합니다.
     8. 완성된 된장찌개와 함께 즐깁니다.

이 레시피로 맛있는 소고기 된장찌개와 명란젓을 즐기시기 바랍니다! 추가적인 질문이 있으시면 언제든지 말씀해 주세요.

## 면접질문 생성 JSON 출력


In [23]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}
"""

    user_message = f"""
채용공고:
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [24]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""
output = job_interview(job_posting)
print(type(output))
print(output)

<class 'str'>
{
    "hard_skill": [
        {
            "question": "Python을 사용하여 머신러닝 모델의 배포를 위해 어떤 절차와 도구를 활용하시겠습니까?",
            "answer": "머신러닝 모델을 배포하기 위해 첫째, 모델을 학습하고 평가한 후, 모델 파일을 저장합니다. 그 다음, Flask 또는 FastAPI와 같은 웹 프레임워크를 사용하여 API를 구축하고, Docker를 활용하여 애플리케이션을 컨테이너화합니다. 이후 Kubernetes를 사용하여 컨테이너를 오케스트레이션하고, 실제 환경에 적합한 CI/CD 파이프라인을 구축하기 위해 Jenkins와 ArgoCD를 활용합니다."
        },
        {
            "question": "CI/CD 파이프라인을 구축할 때 어떤 도구를 선호하며, 그 이유는 무엇인가요?",
            "answer": "저는 GitLab CI/CD와 Jenkins를 주로 사용합니다. GitLab은 프로젝트 관리와 CI/CD를 통합할 수 있어 매우 효율적입니다. Jenkins는 다양한 플러그인을 제공하여 커스터마이징이 용이하고, 오랜 사용 경험으로 인하여 안정성이 뛰어납니다. 두 도구 모두 서버와 클라우드 환경에서 쉽게 통합될 수 있기 때문에 제 선택에 부합합니다."
        },
        {
            "question": "MLflow를 사용할 때 어떤 기능을 가장 많이 활용하게 되나요?",
            "answer": "MLflow의 Tracking 기능을 주로 활용하여 모델 트레이닝 과정에서 발생하는 메트릭, 파라미터 및 아티팩트를 기록합니다. 모델 레지스트리 기능을 통해 모델 버전을 관리하고, 다양한 실험 결과를 비교하는 데 사용합니다. 또한, MLflow UI를 통해 모델의 각 버전과 관련된 정보를 시각적으로 쉽게 확인할 수 있어 매우 유용합니다."
        }
    

In [25]:
import json

# 응답데이터가 json형식이 아니라면 오류발생!
data = json.loads(output)
print(type(data))

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']
print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
[{'question': 'Python을 사용하여 머신러닝 모델의 배포를 위해 어떤 절차와 도구를 활용하시겠습니까?', 'answer': '머신러닝 모델을 배포하기 위해 첫째, 모델을 학습하고 평가한 후, 모델 파일을 저장합니다. 그 다음, Flask 또는 FastAPI와 같은 웹 프레임워크를 사용하여 API를 구축하고, Docker를 활용하여 애플리케이션을 컨테이너화합니다. 이후 Kubernetes를 사용하여 컨테이너를 오케스트레이션하고, 실제 환경에 적합한 CI/CD 파이프라인을 구축하기 위해 Jenkins와 ArgoCD를 활용합니다.'}, {'question': 'CI/CD 파이프라인을 구축할 때 어떤 도구를 선호하며, 그 이유는 무엇인가요?', 'answer': '저는 GitLab CI/CD와 Jenkins를 주로 사용합니다. GitLab은 프로젝트 관리와 CI/CD를 통합할 수 있어 매우 효율적입니다. Jenkins는 다양한 플러그인을 제공하여 커스터마이징이 용이하고, 오랜 사용 경험으로 인하여 안정성이 뛰어납니다. 두 도구 모두 서버와 클라우드 환경에서 쉽게 통합될 수 있기 때문에 제 선택에 부합합니다.'}, {'question': 'MLflow를 사용할 때 어떤 기능을 가장 많이 활용하게 되나요?', 'answer': 'MLflow의 Tracking 기능을 주로 활용하여 모델 트레이닝 과정에서 발생하는 메트릭, 파라미터 및 아티팩트를 기록합니다. 모델 레지스트리 기능을 통해 모델 버전을 관리하고, 다양한 실험 결과를 비교하는 데 사용합니다. 또한, MLflow UI를 통해 모델의 각 버전과 관련된 정보를 시각적으로 쉽게 확인할 수 있어 매우 유용합니다.'}]
[{'question': '팀 내 기술 지원을 제공할 때 어떤 접근 방식을 취하시나요?', 'answer': '저는 문제를 명확히 이해하기 위해 먼저 질문하고 피드백을 선호합니다. 동료들과의 의사소통을 통해 그들이 겪고 있는 문제의 근본 원인을 파악하고, 필요한 경우